<a href="https://colab.research.google.com/github/rodrigospereira22/ProjetoEspectroAutista/blob/main/EspectroAutista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto Posso estar no Espectro?

Projeto para saber a possibilidade da pessoa estar no espectro autista e tirar duvidas sobre o assunto


In [3]:
!pip install -q -U google-generativeai

In [4]:
import google.generativeai as genai
import textwrap

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

Gera a conexao com o genai

In [5]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
from google.colab import userdata


GOOGLE_API_KEY=userdata.get('api_key')

genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(model_name='gemini-1.0-pro')

Ensina e Configura o prompt para gerar as questões que serão exibidas na tela, é montado um padrão para utilizar uma expressão regular


In [6]:
# Defina o prompt
prompt = """**Pergunta 1:** Você tem ou já teve prejuízos na comunicação de maneira geral, como não saber como se expressar, ou não saber o que falar no contexto social?

* (A) Concordo plenamente(80%)
* (B) Concordo um pouco (50%)
* (C) Discordo um pouco (15%)
* (D) Discordo plenamente (0%)

**Pergunta 2:** ocê tem dificuldade ou sente-se constrangido(a) em manter contato visual (curto ou longo prazo)?

* (A) Concordo plenamente(80%)
* (B) Concordo um pouco (50%)
* (C) Discordo um pouco (15%)
* (D) Discordo plenamente (0%)

O padrao acima é só um modelo de como voce deve responder, comece a gerar da pergunta 1 e gere 10 perguntas.
Essas perguntas deve ser como se fosse um psicoterapeuta perguntando,  e as probabilidades percentuais de cada resposta corresponder ao espectro do autismo."""

# Gere o conteúdo usando generate_text com o prompt
texto_resposta = model.generate_content(prompt)

Lê a resposta gerada, faz um laço para ir exibindo e pontuando no final

In [ ]:
import re
perguntas = []
# Encontrar cada bloco de pergunta

total_probabilidade = 0
for match in re.finditer(r"\*\*Pergunta (\d+):\*\*(.*)\n(?:\s*\* \([A-E]\) (.*?) \((.*?)%\)\n)*", texto_resposta.text):
    pergunta_dict = {
        "numero": match.group(1),
        "texto": match.group(2),
        "opcoes": []
    }

    # Dividir o texto da pergunta em linhas
    linhas_opcoes = match.group(0).strip().split('\n')

    # Extrair informações de cada linha de opção
    for linha in linhas_opcoes:
        if linha.startswith("* ("):
            letra, texto, porcentagem = re.match(r"\* \(([A-E])\) (.*) \((.*?)%\)", linha).groups()
            pergunta_dict["opcoes"].append({
                "letra": letra,
                "texto": texto.strip(),
                "probabilidade": int(porcentagem) / 100
            })

    perguntas.append(pergunta_dict)

# Apresentar as perguntas e obter input do usuário
for pergunta in perguntas:
    print(f"Pergunta {pergunta['numero']}: {pergunta['texto']}")
    for opcao in pergunta["opcoes"]:
        print(f"  {opcao['letra']}. {opcao['texto']}")

    resposta_usuario = input("Sua resposta (A-D): ")

    # ... (validação de input opcional) ...

    # Encontrar a resposta completa correspondente à escolha do usuário
    resposta_escolhida = next((opcao for opcao in pergunta["opcoes"] if opcao["letra"] == resposta_usuario.upper()), None)


    if resposta_escolhida:
        print(f"\nVocê escolheu: {resposta_escolhida['texto']} ({resposta_escolhida['probabilidade'] * 100:.0f}%)")
    else:
        print("\nResposta inválida.")

    if resposta_escolhida:
        total_probabilidade += resposta_escolhida["probabilidade"]
   # else:
        #print("\nResposta inválida. Assumindo probabilidade 0 para esta pergunta.")

    print()  # Adicionar uma linha em branco entre as perguntas

media_probabilidade = total_probabilidade / len(perguntas)

# Exibir a probabilidade estatística
print(f"\nProbabilidade estatística de autismo: {media_probabilidade * 100:.2f}%")

Pergunta 1:  Você sente dificuldade em entender as expressões faciais e tons de voz das outras pessoas?
  A. Concordo plenamente
  B. Concordo um pouco
  C. Discordo um pouco
  D. Discordo plenamente
Sua resposta (A-E): a

Você escolheu: Concordo plenamente (80%)

Pergunta 2:  Você tem dificuldade em compreender o sarcasmo ou piadas que dependem do contexto social?
  A. Concordo plenamente
  B. Concordo um pouco
  C. Discordo um pouco
  D. Discordo plenamente
Sua resposta (A-E): b

Você escolheu: Concordo um pouco (50%)

Pergunta 3:  Você prefere atividades solitárias ou tem dificuldade em se envolver em atividades sociais?
  A. Concordo plenamente
  B. Concordo um pouco
  C. Discordo um pouco
  D. Discordo plenamente
Sua resposta (A-E): c

Você escolheu: Discordo um pouco (15%)

Pergunta 4:  Você tem dificuldade em se adaptar a mudanças nas rotinas ou horários?
  A. Concordo plenamente
  B. Concordo um pouco
  C. Discordo um pouco
  D. Discordo plenamente
Sua resposta (A-E): a

Você e

Faz o modelo gerar um aviso para apresentar no final do questionario

In [ ]:
prompt = "Gere um aviso que um questionario respondido para saber se uma pessoa é autista não é preciso e em caso de duvidas deve procurar um profissional qualificado"
aviso = model.generate_content(prompt)
display(to_markdown(aviso.text))


> **Aviso Importante**
> 
> O questionário que você respondeu não é um diagnóstico confiável de autismo.
> 
> Os resultados deste questionário não devem ser interpretados como um diagnóstico definitivo.
> 
> Se você tiver preocupações sobre possíveis sintomas de autismo, é crucial procurar orientação profissional de um médico, psicólogo ou outro profissional de saúde qualificado.
> 
> Os profissionais de saúde podem fornecer avaliações abrangentes e diagnósticos precisos, levando em consideração todos os aspectos relevantes, incluindo histórico de desenvolvimento, observações clínicas e informações de várias fontes.